In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lower_Grade_Glioma/GSE35158'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling of lower-grade diffuse astrocytic glioma"
!Series_summary	"Diffuse gliomas represent the most prevalent class of primary brain tumor.  Despite significant recent advances in the understanding of glioblastoma (WHO IV), its most malignant subtype, lower-grade (WHO II and III) glioma variants remain comparatively understudied, especially in light of their notably variable clinical behavior.  To examine the foundations of this heterogeneity, we performed multidimensional molecular profiling, including global transcriptional analysis, on 101 lower-grade diffuse astrocytic gliomas collected at our own institution, and validated our findings using publically available gene expression and copy number data from large independent patient cohorts.  We found that IDH mutational status delineated molecularly and clinically distinct glioma subsets, with IDH mutant (IDH mt) tumors exhibiting TP53 mutations, PDGFRA overexpression, and prolong

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

import numpy as np  # Import numpy to define nan

# Step 1: Check gene expression data availability
is_gene_available = True  # From background information, it seems gene expression data is available

# Step 2: Check availability of required variables

# Looking for Lower_Grade_Glioma
for key, values in {0: ['tumor type: normal brain', 'tumor type: diffuse astrocytic glioma'], 1: [np.nan, 'who grade: III', 'who grade: II'], 2: [np.nan, 'subclass: PG', 'subclass: NB', 'subclass: EPL'], 3: [np.nan, 'idh mut status: wt', 'idh mut status: mut'], 4: [np.nan, 'pten methylation: U', 'pten methylation: M', 'pten methylation: NA'], 5: [np.nan, 'cdkn2a methylation: U', 'cdkn2a methylation: NA', 'cdkn2a methylation: M'], 6: [np.nan, 'tp53 seq: mut', 'tp53 seq: wt', 'tp53 seq: NA'], 7: [np.nan, 'pten del: No', 'pten del: NA', 'pten del: Yes'], 8: [np.nan, 'pdgfra ihc: -', 'pdgfra ihc: +'], 9: [np.nan, 'p53 ihc: +', 'p53 ihc: -'], 10: [np.nan, 'p-pras40 ihc: +', 'p-pras40 ihc: -', 'p-pras40 ihc: NA'], 11: [np.nan, 'egfr amp: No', 'egfr amp: Yes', 'egfr amp: NA']}.items():
    if "tumor type: diffuse astrocytic glioma" in values:
        trait_row = key
        break

# Looking for age: not available in given characteristics dictionary
age_row = None

# Looking for gender: not available in given characteristics dictionary
gender_row = None

# Define data type conversion functions
def convert_trait(value):
    try:
        trait = value.split(":")[1].strip()
        if trait == "diffuse astrocytic glioma":
            return 1
        return 0
    except:
        return None

# Age and gender data not available
convert_age = None
convert_gender = None

# Step 3: Save cohort information
save_cohort_info('GSE35158', './preprocessed/Lower_Grade_Glioma/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction - conditionally based on trait_row
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lower_Grade_Glioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lower_Grade_Glioma/trait_data/GSE35158.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM864095': [0], 'GSM864096': [1], 'GSM864097': [1], 'GSM864098': [1], 'GSM864099': [1], 'GSM864100': [1], 'GSM864101': [1], 'GSM864102': [1], 'GSM864103': [1], 'GSM864104': [1], 'GSM864105': [1], 'GSM864106': [1], 'GSM864107': [1], 'GSM864108': [1], 'GSM864109': [1], 'GSM864110': [1], 'GSM864111': [1], 'GSM864112': [1], 'GSM864113': [1], 'GSM864114': [1], 'GSM864115': [1], 'GSM864116': [1], 'GSM864117': [1], 'GSM864118': [1], 'GSM864119': [1], 'GSM864120': [1], 'GSM864121': [1], 'GSM864122': [1], 'GSM864123': [1], 'GSM864124': [1], 'GSM864125': [1], 'GSM864126': [1], 'GSM864127': [1], 'GSM864128': [1], 'GSM864129': [1], 'GSM864130': [1], 'GSM864131': [1], 'GSM864132': [1], 'GSM864133': [1], 'GSM864134': [1], 'GSM864135': [1], 'GSM864136': [1], 'GSM864137': [1], 'GSM864138': [1], 'GSM864139': [1], 'GSM864140': [1], 'GSM864141': [1], 'GSM864142': [1], 'GSM864143': [1], 'GSM864144': [1], 'GSM864145': [1], 'GSM864146': [1], 'GSM864147': [1], 'GSM864148': [1], 'GSM864149': [1], 'GSM86415

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651235',
       'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651254',
       'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651285',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651315', 'ILMN_1651316',
       'ILMN_1651336', 'ILMN_1651343', 'ILMN_1651346', 'ILMN_1651347'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Addres

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Lower_Grade_Glioma/gene_data/GSE35158.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Lower_Grade_Glioma')

# 4. Save the cohort information.
save_cohort_info('GSE35158', './preprocessed/Lower_Grade_Glioma/cohort_info.json', True, True, trait_biased, merged_data)

# Only save the merged data if the trait is not severely biased.
if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Lower_Grade_Glioma/GSE35158.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Lower_Grade_Glioma', the least common label is '0.0' with 1 occurrences. This represents 1.23% of the dataset.
The distribution of the feature 'Lower_Grade_Glioma' in this dataset is severely biased.

